In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from keras.models import load_model

Using TensorFlow backend.


In [3]:
from sklearn.metrics import confusion_matrix

In [4]:
from os import system as sys

In [5]:
from time import sleep

In [6]:
wavelen=3500

In [7]:
def fft_trapezoid(rise_time, flat_top , tau):
    length = wavelen
    length = length+2*rise_time+flat_top;
    p = zeros(length)
    s = zeros(length)
    input2 = zeros(length)
    p[0] = s[0] = input2[0] = 0.;
    for i in range(1,length):
        input2[i] = s[i] = 0.;
    input2[1]=1.;
    tau = 1/(exp(1./tau)-1);
    for i in range(1,length):
        if i>=2*rise_time+flat_top:
            d = input2[i]-input2[i-rise_time]-input2[i-rise_time-flat_top]+input2[i-2*rise_time-flat_top]
        else:
            if i>=rise_time+flat_top:
                d = input2[i]-input2[i-rise_time]-input2[i-rise_time-flat_top]
            else:
                if i>=rise_time:
                    d = input2[i]-input2[i-rise_time]
                else:
                    d = input2[i];
        p[i] = p[i-1]+d;
        s[i] = s[i-1]+p[i]+tau*d;
    for i in range(length):
        s[i] = s[i]/(rise_time*tau);
    
    res = fft.rfft(s)
    return res[:-(flat_top+rise_time)]

In [8]:
def fft_cusp(shape_time, tau):
    length=wavelen
    k=2*shape_time+1;
    length = length+k;  
    m2=1.; 
    m1=m2/(exp(1./tau)-1.);
    p = empty(length)
    q = empty(length)
    s = empty(length)
    input_ = zeros(length)
    p[0]=q[0]=s[0]=input_[0]=0.
    input_[1]=1.
    dk=0.
    dl=0.
    for i in range(1,length):
        dk=0.
        dl=0.
        dk = input_[i]
        if i>=k:
            dk -= input_[i-k]

        if i>=shape_time:
            dl += input_[i-shape_time]

        if i>=shape_time+1:
            dl -= input_[i-shape_time-1]

        
        p[i] = p[i-1]+dk-k*dl;
        q[i] = q[i-1]+m2*p[i];
        s[i] = s[i-1]+q[i]+m1*p[i];
    for i in range(length):
        s[i] = s[i]/(0.5*shape_time*(shape_time+1)*m1);
    res = fft.rfft(s)
    return res[:-1*shape_time];

In [9]:
def format_data(input_X,y):    
    input_shape = (len(input_X),wavelen,4)
    X = empty(input_shape)

    short_trap=fft_trapezoid(10,0,1250)
    long_trap=fft_trapezoid(50,0,1250)
    cusp=fft_cusp(100,1250)
    
    for i in range(len(input_X)):
        X[i][:,0] = input_X[i]
        X[i][:,1] = irfft(long_trap*rfft(input_X[i]))
        X[i][:,2] = irfft(short_trap*rfft(input_X[i]))
        X[i][:,3] = irfft(cusp*rfft(input_X[i]))
    X = X.reshape(len(input_X),wavelen,4)
    return X,y

In [10]:
pwd

'/home/david/Documents/comparison'

In [11]:
model = load_model('../experimental_ml/tmp.h5')

In [12]:
mat = array([[0,0],[0,0]])

In [13]:
for i in range(1,500):
    # generate data
    result = 1
    while(result!=0):
        result = sys('../data_generator/bin/pulse_generation '+str(i)+'.label '+str(i)+'.data')
    
    data = loadtxt(str(i)+'.data')
    X, y = format_data(data, append(zeros(int(len(data)/3)),ones(int(2*len(data)/3))))
    pred = around(model.predict(X, batch_size=64),0)
    mat += confusion_matrix(y,pred)
    
    result = 1
    while(result!=0):
        result = sys('./gdrive-linux-x64 upload -p 1u2WCILwYJewZ8SFTGAyIdvlL80xJLOln '+str(i)+'.label')
    
    result = 1
    while(result!=0):
        result = sys('./gdrive-linux-x64 upload -p 1u2WCILwYJewZ8SFTGAyIdvlL80xJLOln '+str(i)+'.data')

    result = 1
    while(result!=0):
        result = sys('rm '+str(i)+'.data')

    result = 1
    while(result!=0):
        result = sys('rm '+str(i)+'.label')
    print('processed batch number '+str(i))

processed batch number 1
processed batch number 2
processed batch number 3
processed batch number 4
processed batch number 5
processed batch number 6
processed batch number 7
processed batch number 8
processed batch number 9
processed batch number 10
processed batch number 11
processed batch number 12
processed batch number 13
processed batch number 14
processed batch number 15
processed batch number 16
processed batch number 17
processed batch number 18
processed batch number 19
processed batch number 20
processed batch number 21
processed batch number 22
processed batch number 23
processed batch number 24
processed batch number 25
processed batch number 26
processed batch number 27
processed batch number 28
processed batch number 29
processed batch number 30
processed batch number 31
processed batch number 32
processed batch number 33
processed batch number 34
processed batch number 35
processed batch number 36
processed batch number 37
processed batch number 38
processed batch numbe

processed batch number 309
processed batch number 310
processed batch number 311
processed batch number 312
processed batch number 313
processed batch number 314
processed batch number 315
processed batch number 316
processed batch number 317
processed batch number 318
processed batch number 319
processed batch number 320
processed batch number 321
processed batch number 322
processed batch number 323
processed batch number 324
processed batch number 325
processed batch number 326
processed batch number 327
processed batch number 328
processed batch number 329
processed batch number 330
processed batch number 331
processed batch number 332
processed batch number 333
processed batch number 334
processed batch number 335
processed batch number 336
processed batch number 337
processed batch number 338
processed batch number 339
processed batch number 340
processed batch number 341
processed batch number 342
processed batch number 343
processed batch number 344
processed batch number 345
p

In [14]:
with open('mat','w') as file:
    file.write(str(mat))